In [1]:
import os
from pathlib import Path

In [2]:
path_current = os.getcwd()
path_main = str(Path(path_current).parent)

### To main directory

In [3]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [4]:
# load libraries and palette
%run __libraries.ipynb

In [5]:
# load functions
%run __functions.ipynb

In [6]:
# load regions data
path_regions_data = path_main + '/files'
path_change(path_regions_data)

%run regions_data.ipynb

In [7]:
# start stopwatch
start = stopwatch_start()

### Variables

In [8]:
path_files = path_main + '/files'

In [9]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [10]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

File 'economics_data' not found
Empty dictonary 'economics_data' created


In [11]:
prices_2021 = economics_data['prices_2021'] \
                if 'prices_2021' in economics_data.keys() else None

### Back to notebook directory

In [12]:
# back to current path
os.chdir(path_current)

# <font color='#2C8B6D'>II. Цены, инфляция</font>

# Section I. Потребительские цены

## Средние потребительские цены

### Средние потребительские цены на отдельные виды товаров и услуг по Российской Федерации, федеральным округам, субъектам Российской Федерации и обследуемым городам (в 2021 г.)

In [13]:
if prices_2021 is None:
    %run CP-consumers-2021.ipynb

### 9.2. Средние потребительские цены на отдельные виды товаров и услуг по Российской Федерации, федеральным округам, субъектам Российской Федерации и обследуемым городам (в 2022 г.)

### 9.3. Средние потребительские цены на отдельные виды товаров и услуг по Российской Федерации, федеральным округам, субъектам Российской Федерации и обследуемым городам (в 2023 г.)

*Источник: https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_10-2023.xlsx*  
*Единицы измерения: *

*Примечания:*  

#### 2024 year link formula

In [14]:
for i in range(1,13):
    link = f'https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_{str(i).zfill(2)}-2024.xlsx'
    link_next = f'https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_{str(i+1).zfill(2)}-2024.xlsx'
    response = requests.get(link)
    response_next = requests.get(link_next)
    condition = ((response.status_code == 200) & (response_next.status_code == 404))
    if condition:
        pci_counsumers_link = link

In [15]:
pci_counsumers_link

'https://rosstat.gov.ru/storage/mediabank/sred_potreb_cen_04-2024.xlsx'

##### Extract Data 

In [16]:
# read excel file
prices_2022_2023_data = pd.ExcelFile(pci_counsumers_link)

In [17]:
# assign sheet names
prices_2022_2023_data_sheets = prices_2022_2023_data.sheet_names

In [18]:
# remove 'Содержание'
prices_2022_2023_data_sheets = prices_2022_2023_data_sheets[1:]

In [19]:
print(prices_2022_2023_data_sheets)

['01(2022)', '02(2022)', '03(2022)', '04(2022)', '05(2022)', '06(2022)', '07(2022)', '08(2022)', '09(2022)', '10(2022)', '11(2022)', '12(2022)', '01(2023)', '02(2023)', '03(2023)', '04(2023)', '05(2023)', '06(2023)', '07(2023)', '08(2023)', '09(2023)', '10(2023)', '11(2023)', '12(2023)', '01(2024)', '02(2024)', '03(2024)', '04(2024)']


In [20]:
prices_2022_2023_raw_dict = {}
for sheet in prices_2022_2023_data_sheets:
    prices_2022_2023_raw_dict['prices_'+ sheet[:2] + '_' + sheet[3:7]] = \
        prices_2022_2023_data.parse(sheet)

In [21]:
prices_2022_2023_raw_dict.keys()

dict_keys(['prices_01_2022', 'prices_02_2022', 'prices_03_2022', 'prices_04_2022', 'prices_05_2022', 'prices_06_2022', 'prices_07_2022', 'prices_08_2022', 'prices_09_2022', 'prices_10_2022', 'prices_11_2022', 'prices_12_2022', 'prices_01_2023', 'prices_02_2023', 'prices_03_2023', 'prices_04_2023', 'prices_05_2023', 'prices_06_2023', 'prices_07_2023', 'prices_08_2023', 'prices_09_2023', 'prices_10_2023', 'prices_11_2023', 'prices_12_2023', 'prices_01_2024', 'prices_02_2024', 'prices_03_2024', 'prices_04_2024'])

In [22]:
# find number of the month of last year in Rosstat data
full_months = len(prices_2022_2023_data_sheets) // 12
month_number = len(prices_2022_2023_data_sheets) - full_months*12

In [23]:
months_2022_2023 = months_list*full_months + months_list[:month_number]

In [24]:
print(months_2022_2023)

['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January', 'February', 'March', 'April']


In [25]:
prices_2022_2023_dict = {}
zip_ = zip(prices_2022_2023_raw_dict.keys(), months_2022_2023, prices_2022_2023_data_sheets)
for key, month, sheet in zip_:
    prices_2022_2023_dict['prices_' + month.lower() + '_' + sheet[-5:-1]] = \
        transform_prices_9_6(prices_2022_2023_raw_dict[key], month, sheet[-5:-1])

##### Concat DataFrames

In [26]:
prices_2022 = pd.DataFrame(
    columns=prices_2022_2023_dict['prices_january_2022'].columns)
prices_2023 = pd.DataFrame(
    columns=prices_2022_2023_dict['prices_january_2023'].columns)
prices_2024 = pd.DataFrame(
    columns=prices_2022_2023_dict['prices_january_2024'].columns)
for key in prices_2022_2023_dict.keys():
    if key[-4:] == '2022':
        prices_2022 = pd.merge(
            prices_2022.T, prices_2022_2023_dict[key].T,
            on=prices_2022.columns)
        prices_2022.index = prices_2022['key_0']
        prices_2022.index.name = None
        prices_2022 = prices_2022.drop('key_0', axis=1)
        prices_2022 = prices_2022.T
    elif key[-4:] == '2023':
        prices_2023 = pd.merge(
            prices_2023.T, prices_2022_2023_dict[key].T,
            on=prices_2023.columns)
        prices_2023.index = prices_2023['key_0']
        prices_2023.index.name = None
        prices_2023 = prices_2023.drop('key_0', axis=1)
        prices_2023 = prices_2023.T
    elif key[-4:] == '2024':
        prices_2024 = pd.merge(
            prices_2024.T, prices_2022_2023_dict[key].T,
            on=prices_2024.columns)
        prices_2024.index = prices_2024['key_0']
        prices_2024.index.name = None
        prices_2024 = prices_2024.drop('key_0', axis=1)
        prices_2024 = prices_2024.T

In [27]:
prices_2022.columns = [i.lower() for i in prices_2022.columns]
prices_2023.columns = [i.lower() for i in prices_2023.columns]
prices_2024.columns = [i.lower() for i in prices_2024.columns]

In [28]:
prices_2022 = prices_2022.astype(float)
prices_2023 = prices_2023.astype(float)
prices_2024 = prices_2024.astype(float)

In [29]:
prices_2022 = prices_2022.round(2)
prices_2023 = prices_2023.round(2)
prices_2024 = prices_2024.round(2)

In [30]:
prices_2022

,"говядина (кроме бескостного мяса), кг","говядина бескостная, кг","свинина (кроме бескостного мяса), кг","свинина бескостная, кг","баранина (кроме бескостного мяса), кг","куры охлажденные и мороженые, кг","окорочка куриные, кг","мясо индейки, кг","печень говяжья, кг","фарш мясной, кг",...,"аренда индивидуального банковского сейфа, в расчете на месяц","комиссия банка за перевод средств с карты на карту клиенту другого банка на территории россии, услуга","годовая стоимость полиса добровольного страхования жилья от стандартных рисков, полис","годовая стоимость полиса добровольного страхования легкового автомобиля от стандартных рисков (каско), полис","годовая стоимость полиса обязательного страхования гражданской ответственности владельцев транспортных средств (осаго), полис","занятия в плавательных бассейнах, месяц","клубная карта в фитнес-клуб, месяц","повторное получение гербового документа в органах загс, документ","ксерокопирование документа, лист","пользование общественным туалетом, посещение"
January 2022,422.09,581.96,309.41,397.88,498.23,180.59,209.55,420.16,315.99,396.76,...,928.25,140.14,5858.27,57743.64,9306.23,2238.93,2291.80,350.0,7.02,17.29
February 2022,428.51,588.77,306.08,395.90,501.81,179.30,209.63,421.77,319.26,400.20,...,932.24,140.94,5858.17,57674.83,9309.50,2241.24,2298.05,350.0,7.07,17.35
March 2022,458.52,623.08,320.06,407.38,515.30,185.52,217.03,429.93,340.40,418.18,...,942.30,141.06,5905.22,65915.84,9408.73,2253.49,2309.74,350.0,9.21,17.66
April 2022,469.69,640.89,319.63,411.76,528.50,190.85,224.20,438.55,353.42,427.45,...,945.86,139.41,5982.05,68289.81,10029.55,2257.07,2327.04,350.0,9.73,18.01
May 2022,474.23,648.83,316.61,411.33,533.55,189.46,225.86,443.30,359.48,431.33,...,977.60,139.16,5983.40,69710.71,10058.67,2260.08,2344.71,350.0,9.90,18.13
June 2022,475.41,654.48,311.86,405.58,537.66,187.85,226.12,442.30,359.42,434.56,...,989.73,138.98,5960.50,68821.57,10063.73,2259.48,2355.03,350.0,10.02,18.33
July 2022,476.09,656.65,310.84,404.80,540.42,186.02,225.28,442.54,356.59,432.84,...,1161.64,137.38,5955.04,68334.51,10180.78,2273.21,2372.81,350.0,10.13,18.38
August 2022,475.35,657.84,310.54,404.44,548.07,183.51,224.53,443.13,353.79,432.32,...,1178.32,137.20,5960.52,69343.34,10174.03,2281.79,2374.30,350.0,10.17,18.42
September 2022,476.40,657.82,309.88,402.37,549.63,181.57,223.44,444.26,351.58,433.07,...,1179.10,137.29,5973.59,72992.50,11918.52,2311.61,2380.40,350.0,10.17,18.54
October 2022,479.81,660.52,309.56,401.46,549.22,181.65,222.19,445.53,349.31,433.37,...,1179.42,137.40,5964.93,72806.21,12006.35,2318.45,2393.92,350.0,10.20,18.50


In [31]:
prices_2023

,"говядина (кроме бескостного мяса), кг","говядина бескостная, кг","свинина (кроме бескостного мяса), кг","свинина бескостная, кг","баранина (кроме бескостного мяса), кг","куры охлажденные и мороженые, кг","окорочка куриные, кг","мясо индейки, кг","печень говяжья, свиная, кг","фарш мясной, кг",...,"аренда индивидуального банковского сейфа, в расчете на месяц","комиссия банка за перевод средств с карты на карту клиенту другого банка на территории россии, услуга","годовая стоимость полиса добровольного страхования жилья от стандартных рисков, полис","годовая стоимость полиса добровольного страхования легкового автомобиля от стандартных рисков (каско), полис","годовая стоимость полиса обязательного страхования гражданской ответственности владельцев транспортных средств (осаго), полис","занятия в плавательных бассейнах, месяц","клубная карта в фитнес-клуб, месяц","повторное получение гербового документа в органах загс, документ","ксерокопирование документа, лист","пользование общественным туалетом, посещение"
January 2023,485.84,662.63,306.43,394.31,544.27,176.25,216.51,444.59,299.57,425.04,...,1242.40,137.50,7129.54,78184.81,13229.10,2356.51,2422.32,350.0,10.67,19.01
February 2023,487.50,663.91,303.12,391.27,545.16,173.78,214.25,445.82,297.42,424.85,...,1242.47,137.52,7183.79,77717.41,13232.37,2362.60,2436.80,350.0,10.72,19.15
March 2023,488.78,663.50,302.96,390.93,545.25,170.40,212.52,446.98,294.43,425.78,...,1252.20,141.50,7171.44,77494.06,13489.79,2370.27,2423.24,350.0,10.71,19.36
April 2023,490.62,663.99,307.40,395.08,551.19,171.70,212.17,446.38,292.61,426.39,...,1258.02,143.18,7126.73,77325.64,13486.00,2372.96,2440.35,350.0,10.74,19.42
May 2023,493.10,665.45,307.10,398.50,576.35,181.08,216.13,446.86,291.59,426.90,...,1259.57,143.21,7056.62,76729.52,13545.49,2375.36,2438.23,350.0,10.76,19.65
June 2023,494.32,667.36,308.38,401.15,584.63,193.48,222.63,446.61,290.62,428.78,...,1262.65,143.18,7050.34,76548.19,13478.66,2377.63,2434.30,350.0,10.79,19.69
July 2023,496.26,670.77,314.54,406.96,590.31,197.85,227.81,448.54,289.40,432.47,...,1261.56,143.18,7075.37,76682.56,13451.59,2378.83,2445.22,350.0,10.83,19.77
August 2023,499.68,676.64,331.44,431.04,603.52,208.80,237.83,457.93,291.41,441.45,...,1262.71,144.27,7080.95,76055.74,13434.53,2386.34,2454.20,350.0,10.85,19.79
September 2023,506.52,683.69,335.96,439.41,612.73,221.02,248.86,471.87,293.36,447.44,...,1270.16,144.22,7156.84,76113.57,13480.71,2440.68,2493.93,350.0,10.90,20.02
October 2023,512.89,694.97,342.28,447.16,627.58,229.65,259.66,487.55,294.98,455.54,...,1270.59,145.58,7176.97,75030.75,13416.34,2453.58,2570.26,350.0,11.06,20.23


In [32]:
prices_2024

,"говядина (кроме бескостного мяса), кг","говядина бескостная, кг","свинина (кроме бескостного мяса), кг","свинина бескостная, кг","баранина (кроме бескостного мяса), кг","куры охлажденные и мороженые, кг","окорочка куриные, кг","мясо индейки, кг","печень говяжья, свиная, кг","фарш мясной, кг",...,"аренда индивидуального банковского сейфа, в расчете на месяц","комиссия банка за перевод средств с карты на карту клиенту другого банка на территории россии, услуга","годовая стоимость полиса добровольного страхования жилья от стандартных рисков, полис","годовая стоимость полиса добровольного страхования легкового автомобиля от стандартных рисков (каско), полис","годовая стоимость полиса обязательного страхования гражданской ответственности владельцев транспортных средств (осаго), полис","занятия в плавательных бассейнах, месяц","клубная карта в фитнес-клуб, месяц","повторное получение гербового документа в органах загс, документ","ксерокопирование документа, лист","пользование общественным туалетом, посещение"
January 2024,527.89,725.34,342.72,452.61,664.10,221.78,265.44,500.55,310.14,465.28,...,1324.32,147.30,7595.91,69192.88,13450.33,2464.88,2620.56,350.0,11.05,19.51
February 2024,535.26,734.32,343.49,451.58,673.85,219.11,262.94,500.27,313.16,466.11,...,1323.35,147.44,7601.92,66433.09,13598.89,2472.53,2643.29,350.0,11.12,19.60
March 2024,543.55,745.03,342.46,451.95,695.86,218.75,262.55,501.87,316.07,468.75,...,1323.59,148.26,7586.48,64681.96,13641.37,2483.59,2646.33,350.0,11.23,19.66
April 2024,550.04,754.96,340.78,452.27,727.96,217.94,262.59,504.16,318.82,470.25,...,1323.98,149.89,7597.49,61558.76,13864.60,2490.16,2650.34,350.0,11.36,19.93


### Combibnig Prices 

In [33]:
prices = (prices_2021.T
          .join(prices_2022.T, how='outer')
          .join(prices_2023.T, how='outer')
          .join(prices_2024.T, how='outer')).T.copy()

In [34]:
len(prices.T)

657

In [35]:
prices.head()

,"абонентская плата за доступ к сети интернет, месяц","абонентская плата за неограниченный объем местных телефонных соединений, месяц","абонентская плата за пакет услуг сотовой связи, месяц","абонентская плата за радиотрансляционную точку, месяц","абонентская плата за телевизионную антенну, месяц","абонентская плата при комбинированной системе оплаты услуг местной телефонной связи, месяц","авторучка шариковая, шт.","азитромицин, 500 мг, 10 капсул (10 таблеток)","активированный уголь, 250 мг, 10 таблеток","аллохол, 50 таблеток",...,"эналаприл, 5 мг, 10 таблеток","энтеросгель, 225 г","эргоферон, 20 таблеток","эссенциале форте н, 300 мг, 10 капсул","эуфиллин, 150 мг, 10 таблеток","юбка для девочек школьного возраста из полушерстяных тканей, шт.","юбка женская из полушерстяных или смесовых тканей, шт.","яблоки, кг","ягоды замороженные, кг","яйца куриные, 10 шт."
January 2021,562.41,462.82,385.61,175.46,218.05,358.33,19.32,755.49,NaN,69.82,...,28.41,NaN,NaN,197.28,NaN,1196.56,2231.55,108.9,375.16,75.38
February 2021,562.52,462.82,386.95,175.5,218.27,358.37,19.45,748.19,NaN,70.48,...,28.5,NaN,NaN,198.21,NaN,1198.3,2236.49,110.97,376.61,77.13
March 2021,562.31,462.82,386.9,178.13,219.62,358.37,19.57,744.32,NaN,71.24,...,28.6,NaN,NaN,198.33,NaN,1201.07,2242.14,112.38,378.98,81.33
April 2021,566.24,462.82,385.48,179.1,220.03,358.37,19.7,736.71,NaN,71.59,...,28.67,NaN,NaN,200.2,NaN,1203.63,2249.98,113.02,384.04,87.01
May 2021,567.08,462.82,385.48,179.1,220.42,358.37,19.84,730.51,NaN,71.84,...,28.66,NaN,NaN,201.51,NaN,1205.77,2256.4,113.13,387.85,80.67


In [36]:
prices.tail()

,"абонентская плата за доступ к сети интернет, месяц","абонентская плата за неограниченный объем местных телефонных соединений, месяц","абонентская плата за пакет услуг сотовой связи, месяц","абонентская плата за радиотрансляционную точку, месяц","абонентская плата за телевизионную антенну, месяц","абонентская плата при комбинированной системе оплаты услуг местной телефонной связи, месяц","авторучка шариковая, шт.","азитромицин, 500 мг, 10 капсул (10 таблеток)","активированный уголь, 250 мг, 10 таблеток","аллохол, 50 таблеток",...,"эналаприл, 5 мг, 10 таблеток","энтеросгель, 225 г","эргоферон, 20 таблеток","эссенциале форте н, 300 мг, 10 капсул","эуфиллин, 150 мг, 10 таблеток","юбка для девочек школьного возраста из полушерстяных тканей, шт.","юбка женская из полушерстяных или смесовых тканей, шт.","яблоки, кг","ягоды замороженные, кг","яйца куриные, 10 шт."
December 2023,608.17,500.89,543.51,184.3,238.85,371.85,30.45,736.96,16.51,105.59,...,22.72,538.83,547.56,237.75,11.07,1400.27,2501.19,122.06,485.18,132.45
January 2024,609.19,501.97,538.33,176.56,239.57,365.93,30.4,751.92,16.8,108.73,...,23.07,546.04,550.23,237.79,11.39,1398.0,2509.9,126.36,479.97,130.06
February 2024,611.8,519.19,538.21,177.55,239.94,378.52,30.55,751.77,17.12,111.76,...,23.26,554.44,549.0,237.61,11.43,1399.95,2530.78,128.91,482.6,129.17
March 2024,615.11,519.26,538.13,177.37,240.45,378.58,30.67,751.74,17.31,113.44,...,23.57,561.54,549.53,238.67,11.53,1400.77,2544.94,131.34,484.02,125.8
April 2024,627.12,519.26,585.13,177.37,240.8,378.58,30.93,753.66,17.67,113.59,...,23.9,564.53,559.8,239.21,11.74,1410.13,2550.8,135.88,487.96,124.45


### Growth since January 2021

In [37]:
prices_growth = pd.DataFrame()
for column in prices.columns:
    prices_growth = pd.concat(
        [prices_growth, normalized_by_first(prices[column], return_type='df')],
        axis=1)

In [38]:
prices_growth.head()

,"абонентская плата за доступ к сети интернет, месяц","абонентская плата за неограниченный объем местных телефонных соединений, месяц","абонентская плата за пакет услуг сотовой связи, месяц","абонентская плата за радиотрансляционную точку, месяц","абонентская плата за телевизионную антенну, месяц","абонентская плата при комбинированной системе оплаты услуг местной телефонной связи, месяц","авторучка шариковая, шт.","азитромицин, 500 мг, 10 капсул (10 таблеток)","активированный уголь, 250 мг, 10 таблеток","аллохол, 50 таблеток",...,"эналаприл, 5 мг, 10 таблеток","энтеросгель, 225 г","эргоферон, 20 таблеток","эссенциале форте н, 300 мг, 10 капсул","эуфиллин, 150 мг, 10 таблеток","юбка для девочек школьного возраста из полушерстяных тканей, шт.","юбка женская из полушерстяных или смесовых тканей, шт.","яблоки, кг","ягоды замороженные, кг","яйца куриные, 10 шт."
January 2021,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,...,1.000000,NaN,NaN,1.000000,NaN,1.000000,1.000000,1.000000,1.000000,1.000000
February 2021,1.000196,1.0,1.003475,1.000228,1.001009,1.000112,1.006729,0.990337,NaN,1.009453,...,1.003168,NaN,NaN,1.004714,NaN,1.001454,1.002214,1.019008,1.003865,1.023216
March 2021,0.999822,1.0,1.003345,1.015217,1.007200,1.000112,1.012940,0.985215,NaN,1.020338,...,1.006688,NaN,NaN,1.005322,NaN,1.003769,1.004746,1.031956,1.010182,1.078933
April 2021,1.006810,1.0,0.999663,1.020745,1.009080,1.000112,1.019669,0.975142,NaN,1.025351,...,1.009152,NaN,NaN,1.014801,NaN,1.005909,1.008259,1.037833,1.023670,1.154285
May 2021,1.008304,1.0,0.999663,1.020745,1.010869,1.000112,1.026915,0.966935,NaN,1.028932,...,1.008800,NaN,NaN,1.021442,NaN,1.007697,1.011136,1.038843,1.033826,1.070178


#### <font color='#820A22'> Export Dataset to Excel and CSV

In [39]:
saveit_excel(
    data=prices,
    filename='prices-2021-2024-consumers',
    path=path_excel_csv,
    sheet='Цены')

File 'prices-2021-2024-consumers.xlsx' created


In [40]:
saveit_csv(
    data=prices, 
    filename='prices-2021-2024-consumers',
    path=path_excel_csv)

File 'prices-2021-2024-consumers.csv' created


In [41]:
saveit_excel(
    data=prices_growth,
    filename='prices-growth-2021-2024-consumers',
    path=path_excel_csv,
    sheet='Цены')

File 'prices-growth-2021-2024-consumers.xlsx' created


In [42]:
saveit_csv(
    data=prices_growth, 
    filename='prices-growth-2021-2024-consumers',
    path=path_excel_csv)

File 'prices-growth-2021-2024-consumers.csv' created


### Save Datasets

In [43]:
economics_data['prices_consumers'] = prices

In [44]:
economics_data['prices_consumers_growth'] = prices_growth

In [45]:
economics_data['prices_2021_consumers'] = prices_2021

In [46]:
economics_data['prices_2022_consumers'] = prices_2022

In [47]:
economics_data['prices_2023_consumers'] = prices_2023

In [48]:
economics_data['prices_2024_consumers'] = prices_2024

In [49]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

File 'economics_data.pkl' saved


### Execution time

In [50]:
stopwatch_stop(start)

Execution time: 0:09:14
